In [11]:
LETTER_IMAGES_FOLDER = "extracted_letter_images"

In [12]:
import cv2
import imutils
def resize_image_to_fit(image, width, height):
    (h, w) = image.shape[:2]
    if w > h:
        image = imutils.resize(image, width=width)
    else:
        image = imutils.resize(image, height=height)
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))
    return image

In [13]:
import numpy as np
import os
from imutils import paths
data = []
labels = []

for image_file in imutils.paths.list_images(LETTER_IMAGES_FOLDER):
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = resize_image_to_fit(image, 20, 20)
    image = np.expand_dims(image, axis=2)
    label = image_file.split(os.path.sep)[-2]
    data.append(image)
    labels.append(label)

In [31]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [32]:
from sklearn.model_selection import train_test_split
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.25, random_state=0)

In [33]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer().fit(Y_train)
Y_train = lb.transform(Y_train)
Y_test = lb.transform(Y_test)

In [34]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
num_classes = 32
model = Sequential()
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(500, activation="relu"))
model.add(Dense(num_classes, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 20, 20, 20)        520       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 20)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 50)        25050     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 50)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1250)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               625500    
_________________________________________________________________
dense_4 (Dense)              (None, 32)                16032     
Total para

In [36]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=10, verbose=1)

Train on 29058 samples, validate on 9686 samples
Epoch 1/1
29058/29058 [==============================] - 58s 2ms/step - loss: 0.0669 - acc: 0.9850 - val_loss: 0.0192 - val_acc: 0.9946


In [38]:
from keras.models import save_model
import pickle
model.save("CAPTCHA_NN.model")
with open("labels", "wb") as f:
    pickle.dump(lb, f)

Prediction

In [3]:
import pickle
from keras.models import load_model

with open("labels", "rb") as f:
    lb = pickle.load(f)
model = load_model("CAPTCHA_NN.model")

In [4]:
CAPTCHA = "captcha_images\\2CXM.png"

In [5]:
def preprocessImage(img):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    return gray, thresh

def contoursToRectangles(contours):
    letter_image_regions = []
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if w / h > 1.25:
            half_width = int(w / 2)
            letter_image_regions.append((x, y, half_width, h))
            letter_image_regions.append((x + half_width, y, half_width, h))
        else:
            letter_image_regions.append((x, y, w, h))
    return letter_image_regions

In [19]:
image = cv2.imread(CAPTCHA)
gray, preprocessedImage = preprocessImage(image)
contours = cv2.findContours(preprocessedImage.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0]
letter_image_regions = contoursToRectangles(contours)
letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])
predictions = []
for letter_bounding_box in letter_image_regions:
    x, y, w, h = letter_bounding_box
    letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]
    letter_image = resize_image_to_fit(letter_image, 20, 20)
    letter_image = np.expand_dims(letter_image, axis=2)
    letter_image = np.expand_dims(letter_image, axis=0)
    prediction = model.predict(letter_image)
    letter = lb.inverse_transform(prediction)[0]
    predictions.append(letter)

In [20]:
predicted_captcha_text = "".join(predictions)
print("Predicted CAPTCHA text is: {}".format(predicted_captcha_text))
print("CAPTCHA text is: {}".format(CAPTCHA.split("\\")[-1].split(".")[0]))

Predicted CAPTCHA text is: 2CXM
CAPTCHA text is: 2CXM
